<a href="https://colab.research.google.com/github/alicewoo0925/ECG-Apnoea-Detection/blob/main/5thmodel/5th_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* minmax scaler
* smote
* top 5 important features: HF, VLF, log(RRseg_std), LF, TP
* stacked classifiers: random forest, svm
* linear regression metaclassifier

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE
import plotly
import plotly.express as px

# Import Dataset from csv file created in MATLAB

[import reference of the UCI dataset]

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')
files = files.upload()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving T_Test.csv to T_Test.csv
Saving T_train.csv to T_train.csv
Saving X_test.csv to X_test.csv
Saving X_train.csv to X_train.csv


In [ ]:
feature_cols = ["RRseg_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI"] #name of the features
result_cols = ["A","N"] #classification results of the training set #used one-hot coding
cols = ["RRseg_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI","A"] #name of the features

In [ ]:
# training set
X_train_df = pd.read_csv("X_train.csv", names=feature_cols)
T_train_df = pd.read_csv("T_train.csv", names=result_cols)

# concatenate features and target
train_df = pd.concat([X_train_df,T_train_df], axis=1)
del train_df['N'] # drop the second element in target (e.g. [0 1] to 0 and [1 0] to 1)
train_df

,RRseg_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI,A
0,1.01390,-2.2262,-2.2082,0.108880,5.0000,0
1,0.94845,-1.3447,-1.2345,0.288230,6.2500,0
2,0.98738,-1.5966,-1.4929,0.223310,7.1429,0
3,1.02000,-3.0473,-3.3568,0.034641,8.3333,0
4,0.97656,-1.3232,-1.2073,0.296360,8.3333,0
...,...,...,...,...,...,...
22648,0.80480,-3.8387,-4.5533,0.010462,2.1739,0
22649,0.74838,-3.3904,-4.5298,0.010733,3.8462,0
22650,0.78724,-4.2226,-4.6701,0.009310,2.0833,0
22651,0.77346,-4.6699,-4.7238,0.008827,1.4706,0


In [ ]:
# testing set
feature_cols = ["RRseg_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI"] #name of the features

X_test_df = pd.read_csv("X_test.csv", names=feature_cols)
X_test_df

,RRseg_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI
0,1.03770,-1.9308,-1.8459,0.156520,7.1429
1,1.04550,-1.7437,-1.5828,0.203620,5.5556
2,0.94048,-1.5096,-1.5136,0.218350,16.6670
3,0.98295,-1.5251,-1.5391,0.212780,10.0000
4,0.90269,-1.3935,-1.4431,0.234390,10.0000
...,...,...,...,...,...
23177,1.05250,-3.3593,-3.5405,0.028754,5.0000
23178,1.05560,-3.2290,-3.3152,0.036130,7.1429
23179,0.89404,-2.0809,-3.9122,0.019954,8.3333
23180,1.02640,-2.8542,-3.7871,0.022517,8.3333


# Data cleaning

In [ ]:
train_df[train_df.isna().any(axis=1)]

,RRseg_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI,A


In [ ]:
# replace NaN with 0
X_test_df.replace(np.nan, 0, inplace = True)
X_test_df.replace([np.inf, -np.inf], 0, inplace = True)

In [ ]:
X_test_df[X_test_df.isna().any(axis=1)]

,RRseg_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI


In [ ]:
# plot histogram of the count of each target
fig = px.histogram(train_df,x='A', title = 'Total Count of Each Target')
fig.show()

In [ ]:
# plot histogram of the count of each target
fig = px.histogram(X_test_df,x='A', title = 'Total Count of Each Target')
fig.show()

# Rescale and oversampling

train set

In [ ]:
X_train = train_df[train_df.columns[:-1]].values
Y_train = train_df[train_df.columns[-1]].values

# rescale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# oversampling
smote = SMOTE(k_neighbors=8, random_state=42)
X_train, Y_train = smote.fit_resample(X_train,Y_train)

data_train = np.hstack((X_train, np.reshape(Y_train, (-1, 1))))

X_train_rescaled = pd.DataFrame(X_train, columns = X_train_df.columns)
X_train_rescaled

,RRseg_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI
0,0.516326,0.696556,0.614323,0.139486,0.085246
1,0.459185,0.872516,0.801151,0.379168,0.110656
2,0.493173,0.822233,0.751571,0.292409,0.128807
3,0.521652,0.532652,0.393937,0.040274,0.153005
4,0.483726,0.876808,0.806370,0.390032,0.153005
...,...,...,...,...,...
33457,0.616410,0.775393,0.735904,0.268144,0.237705
33458,0.570069,0.521248,0.376860,0.036251,0.114440
33459,0.446768,0.757427,0.740146,0.274750,0.088463
33460,0.414479,0.350683,0.175694,0.008807,0.043817


test set (only rescaling)

In [ ]:
X_test = X_test_df[X_test_df.columns[:]].values

# rescale
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

X_test_rescaled = pd.DataFrame(X_test, columns = X_test_df.columns)
X_test_rescaled

,RRseg_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI
0,0.554920,0.629206,0.935943,0.099694,0.142858
1,0.559091,0.665137,0.943072,0.129694,0.111112
2,0.502930,0.710094,0.944947,0.139076,0.333340
3,0.525642,0.707117,0.944256,0.135529,0.200000
4,0.482722,0.732390,0.946857,0.149293,0.200000
...,...,...,...,...,...
23177,0.562834,0.354874,0.890030,0.018315,0.100000
23178,0.564492,0.379897,0.896134,0.023013,0.142858
23179,0.478096,0.600380,0.879959,0.012710,0.166666
23180,0.548877,0.451874,0.883348,0.014342,0.166666


# Select features

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
selector = RandomForestClassifier(n_estimators=9,max_depth=10,random_state=0)
selector.fit(X_train,Y_train)

ValueError: ignored

In [ ]:
# get importance
feature_selection = selector.feature_importances_

In [ ]:
feature_cols = ["RRseg_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI","VLF","LF","HF","TP"]
importance_series = pd.Series(feature_selection, index=feature_cols)
importance_series.sort_values(ascending=False, inplace=True) # descending order

In [ ]:
# plot feature importances
fig = px.bar(importance_series, title= 'Feature importance using Random Forest')
fig.show()

In [ ]:
df_top5 = X_train_rescaled.loc[:,importance_series.index[0:5]]
df_top5

In [ ]:
test_df_top5 = X_test_rescaled.loc[:,importance_series.index[0:5]]
test_df_top5

# Oversampling

In [ ]:
# smote
X_train = X_train_rescaled.values
smote = SMOTE(k_neighbors=8,random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)



```
# This is formatted as code
```

# Classifier Stacking

In [ ]:
len(X_train)

33462

In [ ]:
len(Y_train)

33462

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier()),
     ('svr', LinearSVC())
]

classifier = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

In [ ]:
classifier = classifier.fit(X_train, Y_train)

In [ ]:
pred = classifier.predict(X_test)

In [ ]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
proba = classifier.predict_proba(X_test)

In [ ]:
proba

array([[0.79963664, 0.20036336],
       [0.76776835, 0.23223165],
       [0.78514989, 0.21485011],
       ...,
       [0.64107345, 0.35892655],
       [0.84511548, 0.15488452],
       [0.82986769, 0.17013231]])

In [ ]:
class_df = pd.DataFrame({'Class':np.transpose(pred)})

In [ ]:
class_df

,Class
0,0
1,0
2,0
3,0
4,0
...,...
23177,0
23178,0
23179,0
23180,0


In [ ]:
prob_df = pd.DataFrame(proba,columns=['A','N'])

In [ ]:
prob_df

,A,N
0,0.799637,0.200363
1,0.767768,0.232232
2,0.785150,0.214850
3,0.810679,0.189321
4,0.787020,0.212980
...,...,...
23177,0.842127,0.157873
23178,0.904989,0.095011
23179,0.641073,0.358927
23180,0.845115,0.154885


In [ ]:
result = pd.concat([class_df, prob_df], axis=1)

In [ ]:
result

,Class,A,N
0,0,0.799637,0.200363
1,0,0.767768,0.232232
2,0,0.785150,0.214850
3,0,0.810679,0.189321
4,0,0.787020,0.212980
...,...,...,...
23177,0,0.842127,0.157873
23178,0,0.904989,0.095011
23179,0,0.641073,0.358927
23180,0,0.845115,0.154885


In [ ]:
result.to_csv('result.csv',index=False)